In [1]:
import importlib
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

import corners_edge as ce
importlib.reload(ce)

BASE = Path.cwd()
ARTIFACT_PREFIX = BASE / "corners_model" / "corners_model"
INPUT_CSV  = BASE / "data" / "matchday_inputs.csv"
OUTPUT_CSV = BASE / "data" / "matchday_quotes.csv"

BANKROLL = 50.0
KELLY_FRACTION = 0.5
# --- Bet gating thresholds ---
MIN_EV = 0.02         
MIN_EDGE_PCT = 3.0     
MIN_KELLY = 0.002      
MIN_STAKE_DOLLARS = 1  
MIN_PRICE = 1.0       
EPS = 1e-12

INPUT_CSV.parent.mkdir(parents=True, exist_ok=True)

draws, team_names, div_names, zstats = ce.load_artifacts(ARTIFACT_PREFIX)
print("Artifacts loaded. Teams:", len(team_names), "Divs:", len(div_names))


Artifacts loaded. Teams: 160 Divs: 5


In [2]:
inp = pd.read_csv(INPUT_CSV, sep=None, engine="python", dtype=str).fillna("")
inp = inp.applymap(lambda x: x.strip() if isinstance(x, str) else x)

MISSING = {"", "na", "n/a", "nan", "-", "--"}
for c in ["book_over","book_under","book_exact","odds_home","odds_draw","odds_away","odds_over25","odds_under25"]:
    if c in inp.columns:
        inp[c] = inp[c].apply(lambda s: "" if str(s).strip().lower() in MISSING else s)

def to_float_safe(x):
    try: return float(x)
    except Exception: return np.nan

if "corners_line" in inp.columns:
    inp["corners_line"] = inp["corners_line"].apply(to_float_safe)
if "exact_total" in inp.columns:
    inp["exact_total"] = inp["exact_total"].apply(to_float_safe)

display(inp.head())


/var/folders/xy/fs1cvf415szgw9x79lyywp1h0000gn/T/ipykernel_89001/2825926424.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  inp = inp.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,home_team,away_team,division,corners_line,book_over,book_under,book_exact,exact_total,odds_home,odds_draw,odds_away,odds_over25,odds_under25,pmf_plot
0,Aston Villa,Fulham,E0,10.0,11/10,11/10,7/1,10.0,,,,,,
1,Newcastle,Arsenal,E0,10.0,11/10,1/1,7/1,10.0,,,,,,
2,Everton,West Ham,E0,10.0,6/5,1/1,7/1,10.0,,,,,,
3,Rayo Vallecano,Sevilla,SP1,9.0,10/11,11/8,13/2,9.0,,,,,,
4,Elche,Celta Vigo,SP1,9.0,6/5,1/1,13/2,9.0,,,,,,


In [3]:
def _f(x, default=None):
    try: return float(x)
    except Exception: return default

def normalize_feature_odds(row):
    oh = ce.parse_odds_decimal(row.get("odds_home"))
    od = ce.parse_odds_decimal(row.get("odds_draw"))
    oa = ce.parse_odds_decimal(row.get("odds_away"))
    o25 = ce.parse_odds_decimal(row.get("odds_over25"))
    u25 = ce.parse_odds_decimal(row.get("odds_under25"))
    if any(v is None for v in [oh, od, oa]):
        p_home = float(zstats.mean_p_home); p_draw = (1.0 - p_home)/2.0; p_away = p_draw
        oh, od, oa = 1.0/p_home, 1.0/p_draw, 1.0/p_away
    if any(v is None for v in [o25, u25]):
        p_over = float(zstats.mean_p_over25); p_under = 1.0 - p_over
        o25, u25 = 1.0/p_over, 1.0/p_under
    return oh, od, oa, o25, u25

def breakeven_probs(p_push: float, price_over, price_under):
    import numpy as np
    be_o = (1 - p_push) / price_over  if price_over  and price_over  > 1.0 else np.nan
    be_u = (1 - p_push) / price_under if price_under and price_under > 1.0 else np.nan
    return be_o, be_u


In [4]:
rows = []
for _, r in inp.iterrows():
    home = str(r.get("home_team","")).strip()
    away = str(r.get("away_team","")).strip()
    div  = str(r.get("division","")).strip() or None
    line = _f(r.get("corners_line"), 9.5)

    price_over  = ce.parse_odds_decimal(r.get("book_over"))
    price_under = ce.parse_odds_decimal(r.get("book_under"))
    price_exact = ce.parse_odds_decimal(r.get("book_exact"))
    exact_total = _f(r.get("exact_total"))

    oh, od, oa, o25, u25 = normalize_feature_odds(r)

    res = ce.predict_game(
        draws, team_names, div_names, zstats,
        home_team=home, away_team=away, division=div,
        odds_home=oh, odds_draw=od, odds_away=oa,
        odds_over25=o25, odds_under25=u25,
        total_corners_line=line,
        price_over=price_over, price_under=price_under,
        n_sims=15000
    )
    expected = float(res["expected_corners"])

    p_home, p_draw, p_away = ce.implied_probs_1x2_from_odds(oh, od, oa)
    p_over25 = ce.implied_prob_over25_from_odds(o25, u25)
    z_ph = float(np.clip(ce.zscore(p_home,  zstats.mean_p_home,  zstats.std_p_home),  -6.0, 6.0))
    z_po = float(np.clip(ce.zscore(p_over25, zstats.mean_p_over25, zstats.std_p_over25), -6.0, 6.0))

    def idx(name, universe):
        try: return universe.index(name)
        except ValueError: return 0
    hi = idx(home, team_names); ai = idx(away, team_names)
    if isinstance(div, str) and len(div_names) > 0:
        try: di = div_names.index(div)
        except ValueError: di = 0
    else:
        di = 0

    intercept = draws["intercept"].reshape(-1)
    home_adv  = draws["home_adv"].reshape(-1)
    team_home_eff = draws["team_home_eff"]
    team_away_eff = draws["team_away_eff"]
    alpha = draws["alpha"].reshape(-1)
    beta_homeprob = draws.get("beta_homeprob", np.zeros_like(intercept)).reshape(-1)
    beta_over25   = draws.get("beta_over25",   np.zeros_like(intercept)).reshape(-1)
    div_term = draws["div_eff"][:, di].reshape(-1) if ("div_eff" in draws and len(div_names)>0) else 0.0

    eta = (intercept + home_adv + team_home_eff[:, hi] + team_away_eff[:, ai]
           + div_term + beta_homeprob * z_ph + beta_over25 * z_po)
    mu = np.exp(eta); mu = np.clip(mu, 1e-6, 60.0)

    sims = ce.nb_predictive_samples(mu, alpha, n_sims=35000)
    pmf = ce.pmf_from_draws(sims, vmax=int(np.clip(np.percentile(sims, 99.5), 20, 60)))

    p_over, p_under, p_push = ce.prob_over_under_push(pmf, line)

    ev_over,  k_over  = ce.ev_kelly_two_way(p_over,  p_under, price_over,  KELLY_FRACTION)
    ev_under, k_under = ce.ev_kelly_two_way(p_under, p_over,  price_under, KELLY_FRACTION)

    be_over  = (1 - p_push) / price_over  if price_over  and price_over  > 1.0 else np.nan
    be_under = (1 - p_push) / price_under if price_under and price_under > 1.0 else np.nan
    edge_over  = (p_over  - be_over)  * 100 if np.isfinite(be_over)  else np.nan
    edge_under = (p_under - be_under) * 100 if np.isfinite(be_under) else np.nan


    ev_over_eff  = ev_over  if (price_over  and np.isfinite(ev_over))  else -np.inf
    ev_under_eff = ev_under if (price_under and np.isfinite(ev_under)) else -np.inf
    
    best_side  = "over" if ev_over_eff >= ev_under_eff else "under"
    best_ev    = ev_over_eff  if best_side == "over"  else ev_under_eff
    best_kelly = k_over       if best_side == "over"  else k_under
    best_edge  = edge_over    if best_side == "over"  else edge_under
    best_price = price_over   if best_side == "over"  else price_under
    
    final_kelly_over  = 0.0
    final_kelly_under = 0.0
    take_side = "no_bet"
    no_bet_reason = None
    
    passes = True
    reasons = []
    if not (best_ev > MIN_EV + EPS):
        passes = False; reasons.append("EV<threshold")
    if not (best_kelly > MIN_KELLY + EPS):
        passes = False; reasons.append("Kelly<threshold")
    if not (np.isfinite(best_edge) and (best_edge >= MIN_EDGE_PCT - 1e-9)):
        passes = False; reasons.append("Edge<threshold")
    if not (best_price and best_price > MIN_PRICE):
        passes = False; reasons.append("Price_invalid")
    
    if passes:
        if best_side == "over":
            final_kelly_over = max(0.0, best_kelly)
            take_side = "over"
        else:
            final_kelly_under = max(0.0, best_kelly)
            take_side = "under"
    
    stake_over  = BANKROLL * final_kelly_over
    stake_under = BANKROLL * final_kelly_under
    
    if take_side != "no_bet" and (max(stake_over, stake_under) < MIN_STAKE_DOLLARS):
        # zero out tiny bets
        final_kelly_over = final_kelly_under = 0.0
        stake_over = stake_under = 0.0
        take_side = "no_bet"
        reasons.append("Stake<threshold")
    
    if take_side == "no_bet":
        no_bet_reason = ",".join(reasons) if reasons else "no_edge"
    
    assert (final_kelly_over == 0.0) or (final_kelly_under == 0.0)

    fair_exact = None; ev_exact = None; kelly_exact = None; p_exact = None
    if not pd.isna(exact_total):
        p_exact = pmf.get(int(round(exact_total)), 0.0)
        fair_exact = (1.0 / p_exact) if p_exact and p_exact > 0 else None
        if price_exact and fair_exact:
            ev_exact, kelly_exact = ce.ev_kelly_single(p_exact, price_exact, KELLY_FRACTION)

    rows.append({
        "home_team": home, "away_team": away, "division": div,
        "corners_line": float(line) if line is not None else None,
        "expected_corners": round(expected, 3) if expected is not None else None,
        "p_over": round(p_over, 4),
        "p_under": round(p_under, 4),
        "p_push": round(p_push, 4),
        "book_over": price_over, "book_under": price_under,
        "fair_over": round(1.0/max(p_over,1e-9), 3),
        "fair_under": round(1.0/max(p_under,1e-9), 3),
        "EV_over_per_$": round(ev_over, 4) if np.isfinite(ev_over) else None,
        "EV_under_per_$": round(ev_under, 4) if np.isfinite(ev_under) else None,
        "kelly_over": round(final_kelly_over, 4),
        "kelly_under": round(final_kelly_under, 4),
        "stake_over": round(stake_over, 2),
        "stake_under": round(stake_under, 2),
        "take_side": take_side,
        "no_bet_reason": no_bet_reason,
        "breakeven_over": round(be_over, 4) if np.isfinite(be_over) else None,
        "breakeven_under": round(be_under, 4) if np.isfinite(be_under) else None,
        "edge_over_pct": round(edge_over, 2) if np.isfinite(edge_over) else None,
        "edge_under_pct": round(edge_under, 2) if np.isfinite(edge_under) else None,
        "exact_total": exact_total, "book_exact": price_exact,
        "p_exact": (round(p_exact,4) if p_exact is not None else None),
        "fair_exact": (round(fair_exact,3) if fair_exact is not None else None),
        "EV_exact_per_$": (round(ev_exact,4) if ev_exact is not None else None),
        "kelly_exact": (round(kelly_exact,4) if kelly_exact is not None else None),
    })

quotes = pd.DataFrame(rows)

quotes["raw_stake"] = quotes[["stake_over", "stake_under"]].max(axis=1)

total_raw = quotes["raw_stake"].sum()
if total_raw > 0:
    target_fraction = 0.8
    target_total = BANKROLL * target_fraction
    scale = min(1.0, target_total / total_raw)

    alloc = (quotes["raw_stake"] * scale).astype(float)

    alloc_rounded = alloc.round(2)
    diff = round(target_total - alloc_rounded.sum(), 2)
    if abs(diff) >= 0.01:
        idx_max = alloc_rounded.idxmax()
        alloc_rounded.loc[idx_max] = max(0.0, alloc_rounded.loc[idx_max] + diff)

    quotes["stake_final"] = alloc_rounded
else:
    quotes["stake_final"] = 0.0

quotes["stake_over"]  = np.where(quotes["stake_over"]  > 0, quotes["stake_final"], 0.0).round(2)
quotes["stake_under"] = np.where(quotes["stake_under"] > 0, quotes["stake_final"], 0.0).round(2)

quotes["kelly_over"]  = np.where(quotes["stake_over"]  > 0, quotes["stake_final"] / BANKROLL, 0.0).round(4)
quotes["kelly_under"] = np.where(quotes["stake_under"] > 0, quotes["stake_final"] / BANKROLL, 0.0).round(4)

quotes = quotes.drop(columns=["raw_stake"], errors="ignore")

print(f"Total final stake: {quotes['stake_final'].sum():.2f}  (Bankroll: {BANKROLL:.2f})")



display(quotes)
OUTPUT_CSV.parent.mkdir(parents=True, exist_ok=True)
quotes.to_csv(OUTPUT_CSV, index=False)
print("Saved quotes to:", OUTPUT_CSV)


Total final stake: 40.00  (Bankroll: 50.00)


,home_team,away_team,division,corners_line,expected_corners,p_over,p_under,p_push,book_over,book_under,...,breakeven_under,edge_over_pct,edge_under_pct,exact_total,book_exact,p_exact,fair_exact,EV_exact_per_$,kelly_exact,stake_final
0,Aston Villa,Fulham,E0,10.0,10.048,0.4204,0.4599,0.1165,2.100000,2.100000,...,0.4207,-0.03,3.92,10.0,8.0,0.1165,8.581,-0.0677,0.0000,1.11
1,Newcastle,Arsenal,E0,10.0,9.782,0.3914,0.4908,0.1154,2.100000,2.000000,...,0.4423,-2.98,4.85,10.0,8.0,0.1154,8.666,-0.0768,0.0000,1.43
2,Everton,West Ham,E0,10.0,10.192,0.4334,0.4455,0.1176,2.200000,2.000000,...,0.4412,3.23,0.43,10.0,8.0,0.1176,8.503,-0.0592,0.0000,0.89
3,Rayo Vallecano,Sevilla,SP1,9.0,9.622,0.4878,0.3896,0.1207,1.909091,2.375000,...,0.3702,2.72,1.94,9.0,7.5,0.1207,8.288,-0.0951,0.0000,0.00
4,Elche,Celta Vigo,SP1,9.0,9.643,0.4949,0.3843,0.1187,2.200000,2.000000,...,0.4406,9.43,-5.63,9.0,7.5,0.1187,8.424,-0.1096,0.0000,2.51
5,Barcelona,Real Sociedad,SP1,10.0,9.665,0.3784,0.5039,0.1156,1.833333,2.375000,...,0.3724,-10.40,13.15,10.0,8.5,0.1156,8.651,-0.0174,0.0000,3.28
6,Real Betis,Osasuna,SP1,9.0,9.510,0.4707,0.4012,0.1267,1.909091,2.375000,...,0.3677,1.33,3.35,9.0,7.5,0.1267,7.892,-0.0496,0.0000,0.84
7,Valencia,Real Oviedo,SP1,11.0,9.859,0.2892,0.6049,0.1039,2.200000,1.909091,...,0.4694,-11.81,13.55,11.0,8.5,0.1039,9.626,-0.1170,0.0000,4.09
8,Sassuolo,Udinese,I1,10.0,10.467,0.4713,0.4113,0.1149,2.250000,1.909091,...,0.4636,7.79,-5.23,10.0,8.0,0.1149,8.704,-0.0809,0.0000,2.04
9,Pisa,Fiorentina,I1,9.0,9.552,0.4775,0.3966,0.1237,2.500000,1.833333,...,0.4780,12.70,-8.14,9.0,7.5,0.1237,8.085,-0.0724,0.0000,3.06


Saved quotes to: /Users/jacksimonson/Documents/Soccer Betting/data/matchday_quotes.csv
